Contains the complete step-wise filter, pull, commpile, and analyze pipeline for the provided git repositories.  
The expected input is a list of repositories of which the data is acquired through the github web API. 

Make sure that a SonarQube instance is running prior to running this script and that the credentials used in the analysis phase are set correctly. 
Consider that executing this script takes quite a while. 

The results are put in the ``./results`` folder

In [21]:
import array 
from git import Repo, GitCommandError
import os
import requests
import subprocess
import shutil
from typing import Tuple

with open("./data/used_repositories.csv", "r") as data_file: 
    data = [entry.strip().split(",") for entry in data_file.readlines()[1:]]

print(f'loaded data:\n{data}')

loaded data:
[['commons-lang', 'https://github.com/apache/commons-lang', '26071', 'master', 'Java'], ['karaf', 'https://github.com/apache/karaf', '67084', 'main', 'Java'], ['jena', 'https://github.com/apache/jena', '422407', 'main', 'Java'], ['jackrabbit-filevault', 'https://github.com/apache/jackrabbit-filevault', '4594', 'master', 'Java'], ['karaf-decanter', 'https://github.com/apache/karaf-decanter', '8363', 'main', 'Java'], ['commons-text', 'https://github.com/apache/commons-text', '2737', 'master', 'Java'], ['tinkerpop', 'https://github.com/apache/tinkerpop', '182945', 'master', 'Java'], ['systemds', 'https://github.com/apache/systemds', '333609', 'main', 'Java'], ['bahir-flink', 'https://github.com/apache/bahir-flink', '1351', 'master', 'Java'], ['rocketmq', 'https://github.com/apache/rocketmq', '11820', 'master', 'Java'], ['servicecomb-pack', 'https://github.com/apache/servicecomb-pack', '15119', 'master', 'Java'], ['myfaces-tobago', 'https://github.com/apache/myfaces-tobago', '

In [22]:
def is_java(entry: array) -> bool: 
    return entry[4] == "Java"

def is_maven(entry: array) -> bool: 
    """Pings project's pom.xml file and returns true if it exists."""

    url = os.path.join("https://raw.githubusercontent.com/apache/", entry[0], entry[3], "pom.xml")
    req = requests.get(url)
    
    return req.status_code == 200

def is_considered(entry: array) -> bool: 
    """Returns true if the project is a maven project"""

    return is_java(entry) and is_maven(entry)

Pulls and compiles maven repositories


In [23]:
def clone_repository(entry: array) -> Tuple[int, str]:
    """
    Clones repository to the ./repos folder and 
    returns the status code and repo's folder.
    """

    url = entry[1]
    dir = os.path.join("./repos", entry[1])

    try:
        Repo.clone_from(url, dir)
        status = 0

    except GitCommandError as e:
        if e.status != 128:
            status = 0
            print(
                f'repository for {entry[0]} could not be cloned with status {e.status}')
        else:
            print(f'repository for {entry[0]} is already cloned')

    finally:
        return status, dir


def compile_maven(entry: array, dir: str) -> int:
    """
    Compiles maven directory and skips tests
    and returns compilation status
    """

    os.chdir(dir)

    args = ['mvn', '-Dmaven.test.skip=true', 'clean', 'install']
    status = subprocess.run(args=args)
    
    os.chdir("../..")

    return status.returncode

Implements SonarQube analysis and data extraction

In [24]:
server_url = "http://localhost:9000"
auth = ('admin', 'password')

def create_sonarqube_project(entry: array) -> Tuple[int, str]: 
    """Creates SonarQube project if none exists yet"""

    url = f"{server_url}/api/projects/create"
    data = {'name':entry[0], 'project':entry[0], 'visibility':'public'}
    c_res = requests.post(url=url, data=data, auth=auth)

    url = f"{server_url}/api/user_tokens/generate"
    data = {'name':entry[0]}
    t_res = requests.post(url=url, data=data, auth=auth)
    token = t_res.json()["token"]

    return c_res.status_code, token


def perform_sonarqube_analysis(entry: array, token:str) -> int:
    """executes sonarqube analaysis and sends it to the server"""

    os.chdir(f"./repos/{entry[0]}")

    args = ['sonar-scanner', f'-Dsonar.projectKey={entry[0]}', f'-Dsonar.host.url={server_url}', f'-Dsonar.login={token}']
    res = subprocess.run(args)

    os.chdir("../..")

    return 0



def export_sonarqube_issues(entry: array):
    pass

def delete_sonarqube_project(entry: array): 
    """Deletes sonarqube project and access credentials"""

    url = f'{server_url}/api/projects/delete'
    data = {'project':entry[0]}
    requests.post(url=url, data=data, auth=auth)

    url = f'{server_url}/api/user_tokens/revoke'
    data = {'name':entry[0]}
    requests.post(url=url, data=data, auth=auth)



Implements the pipeline lifecycle.

In [25]:
def perform_lifecycle(entry: array):
    # step 1: filtering
    # if not is_considered(entry): 
    #     print(f"filtered out {entry[0]}.")
    #     return

    # # step 2: cloning repositories.
    # status, dir = clone_repository(entry)
    # if status != 0: 
    #     print(f'cloning repository failed for {entry[0]} with status {status}')
    #     return 

    # # step 3: compiling maven project. 
    # status = compile_maven(entry, dir)
    # if status != 0: 
    #     print(f"maven compilation failed for {entry[0]} wit status {status}.")
    #     shutil.rmtree(dir)
    #     return 

    # step 4: creating sonarqube project
    status, token = create_sonarqube_project(entry)

    # step 5: running SonarQube
    status = perform_sonarqube_analysis(entry, token)
    if status != 0: 
        print(f'sonarqube analysis failed for {entry[0]} with status {status}')
        return 

    # step 6: extracting sonarqube data
    export_sonarqube_issues(entry)

    # step 7: deleting sonarqube project
    delete_sonarqube_project(entry)
    
    

for entry in data: 
    perform_lifecycle(entry)

INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: SonarScanner 4.6.2.2472
INFO: Java 11.0.11 AdoptOpenJDK (64-bit)
INFO: Linux 5.11.0-46-generic amd64
INFO: User cache: /home/willem/.sonar/cache
INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: Analyzing on SonarQube server 9.2.4
INFO: Default locale: "en_US", source code encoding: "UTF-8" (analysis is platform dependent)
INFO: Load global settings
INFO: Load global settings (done) | time=104ms
INFO: Server id: BF41A1F2-AX4z6Sh5gaWG4x2kXYr7
INFO: User cache: /home/willem/.sonar/cache
INFO: Load/download plugins
INFO: Load plugins index
INFO: Load plugins index (done) | time=61ms
INFO: Load/download plugins (done) | time=159ms
INFO: Process project properties
INFO: Process project properties (done) | time

ERROR: Error during SonarScanner execution
org.sonar.java.AnalysisException: Your project contains .java files, please provide compiled classes with sonar.java.binaries property, or exclude them from the analysis with sonar.exclusions property.
	at org.sonar.java.classpath.ClasspathForMain.init(ClasspathForMain.java:75)
	at org.sonar.java.classpath.AbstractClasspath.getElements(AbstractClasspath.java:321)
	at org.sonar.java.SonarComponents.getJavaClasspath(SonarComponents.java:183)
	at org.sonar.java.JavaFrontend.<init>(JavaFrontend.java:89)
	at org.sonar.plugins.java.JavaSensor.execute(JavaSensor.java:108)
	at org.sonar.scanner.sensor.AbstractSensorWrapper.analyse(AbstractSensorWrapper.java:48)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.execute(ModuleSensorsExecutor.java:85)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.lambda$execute$1(ModuleSensorsExecutor.java:59)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.withModuleStrategy(ModuleSensorsExecutor.java:77)
	at o

INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: SonarScanner 4.6.2.2472
INFO: Java 11.0.11 AdoptOpenJDK (64-bit)
INFO: Linux 5.11.0-46-generic amd64
INFO: User cache: /home/willem/.sonar/cache
INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: Analyzing on SonarQube server 9.2.4
INFO: Default locale: "en_US", source code encoding: "UTF-8" (analysis is platform dependent)
INFO: Load global settings
INFO: Load global settings (done) | time=81ms
INFO: Server id: BF41A1F2-AX4z6Sh5gaWG4x2kXYr7
INFO: User cache: /home/willem/.sonar/cache
INFO: Load/download plugins
INFO: Load plugins index
INFO: Load plugins index (done) | time=40ms
INFO: Load/download plugins (done) | time=127ms
INFO: Process project properties
INFO: Process project properties (done) | time=

ERROR: Error during SonarScanner execution
org.sonar.java.AnalysisException: Your project contains .java files, please provide compiled classes with sonar.java.binaries property, or exclude them from the analysis with sonar.exclusions property.
	at org.sonar.java.classpath.ClasspathForMain.init(ClasspathForMain.java:75)
	at org.sonar.java.classpath.AbstractClasspath.getElements(AbstractClasspath.java:321)
	at org.sonar.java.SonarComponents.getJavaClasspath(SonarComponents.java:183)
	at org.sonar.java.JavaFrontend.<init>(JavaFrontend.java:89)
	at org.sonar.plugins.java.JavaSensor.execute(JavaSensor.java:108)
	at org.sonar.scanner.sensor.AbstractSensorWrapper.analyse(AbstractSensorWrapper.java:48)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.execute(ModuleSensorsExecutor.java:85)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.lambda$execute$1(ModuleSensorsExecutor.java:59)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.withModuleStrategy(ModuleSensorsExecutor.java:77)
	at o

INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: SonarScanner 4.6.2.2472
INFO: Java 11.0.11 AdoptOpenJDK (64-bit)
INFO: Linux 5.11.0-46-generic amd64
INFO: User cache: /home/willem/.sonar/cache
INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: Analyzing on SonarQube server 9.2.4
INFO: Default locale: "en_US", source code encoding: "UTF-8" (analysis is platform dependent)
INFO: Load global settings
INFO: Load global settings (done) | time=111ms
INFO: Server id: BF41A1F2-AX4z6Sh5gaWG4x2kXYr7
INFO: User cache: /home/willem/.sonar/cache
INFO: Load/download plugins
INFO: Load plugins index
INFO: Load plugins index (done) | time=60ms
INFO: Load/download plugins (done) | time=177ms
INFO: Process project properties
INFO: Process project properties (done) | time

...

INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: SonarScanner 4.6.2.2472
INFO: Java 11.0.11 AdoptOpenJDK (64-bit)
INFO: Linux 5.11.0-46-generic amd64
INFO: User cache: /home/willem/.sonar/cache
INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: Analyzing on SonarQube server 9.2.4
INFO: Default locale: "en_US", source code encoding: "UTF-8" (analysis is platform dependent)
INFO: Load global settings
INFO: Load global settings (done) | time=84ms
INFO: Server id: BF41A1F2-AX4z6Sh5gaWG4x2kXYr7
INFO: User cache: /home/willem/.sonar/cache
INFO: Load/download plugins
INFO: Load plugins index
INFO: Load plugins index (done) | time=47ms
INFO: Load/download plugins (done) | time=135ms
INFO: Process project properties
INFO: Process project properties (done) | time=

ERROR: Error during SonarScanner execution
org.sonar.java.AnalysisException: Your project contains .java files, please provide compiled classes with sonar.java.binaries property, or exclude them from the analysis with sonar.exclusions property.
	at org.sonar.java.classpath.ClasspathForMain.init(ClasspathForMain.java:75)
	at org.sonar.java.classpath.AbstractClasspath.getElements(AbstractClasspath.java:321)
	at org.sonar.java.SonarComponents.getJavaClasspath(SonarComponents.java:183)
	at org.sonar.java.JavaFrontend.<init>(JavaFrontend.java:89)
	at org.sonar.plugins.java.JavaSensor.execute(JavaSensor.java:108)
	at org.sonar.scanner.sensor.AbstractSensorWrapper.analyse(AbstractSensorWrapper.java:48)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.execute(ModuleSensorsExecutor.java:85)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.lambda$execute$1(ModuleSensorsExecutor.java:59)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.withModuleStrategy(ModuleSensorsExecutor.java:77)
	at o

INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: SonarScanner 4.6.2.2472
INFO: Java 11.0.11 AdoptOpenJDK (64-bit)
INFO: Linux 5.11.0-46-generic amd64
INFO: User cache: /home/willem/.sonar/cache
INFO: Scanner configuration file: /home/willem/Downloads/sonar-scanner-4.6.2.2472-linux/conf/sonar-scanner.properties
INFO: Project root configuration file: NONE
INFO: Analyzing on SonarQube server 9.2.4
INFO: Default locale: "en_US", source code encoding: "UTF-8" (analysis is platform dependent)
INFO: Load global settings
INFO: Load global settings (done) | time=78ms
INFO: Server id: BF41A1F2-AX4z6Sh5gaWG4x2kXYr7
INFO: User cache: /home/willem/.sonar/cache
INFO: Load/download plugins
INFO: Load plugins index
INFO: Load plugins index (done) | time=42ms
INFO: Load/download plugins (done) | time=132ms
INFO: Process project properties
INFO: Process project properties (done) | time=

ERROR: Error during SonarScanner execution
org.sonar.java.AnalysisException: Your project contains .java files, please provide compiled classes with sonar.java.binaries property, or exclude them from the analysis with sonar.exclusions property.
	at org.sonar.java.classpath.ClasspathForMain.init(ClasspathForMain.java:75)
	at org.sonar.java.classpath.AbstractClasspath.getElements(AbstractClasspath.java:321)
	at org.sonar.java.SonarComponents.getJavaClasspath(SonarComponents.java:183)
	at org.sonar.java.JavaFrontend.<init>(JavaFrontend.java:89)
	at org.sonar.plugins.java.JavaSensor.execute(JavaSensor.java:108)
	at org.sonar.scanner.sensor.AbstractSensorWrapper.analyse(AbstractSensorWrapper.java:48)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.execute(ModuleSensorsExecutor.java:85)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.lambda$execute$1(ModuleSensorsExecutor.java:59)
	at org.sonar.scanner.sensor.ModuleSensorsExecutor.withModuleStrategy(ModuleSensorsExecutor.java:77)
	at o

FileNotFoundError: [Errno 2] No such file or directory: './repos/shardingsphere'